In [1]:
%matplotlib qt5
from process_files import *
from functions import *
from func_flats import *

qt.qpa.xcb: X server does not support XInput 2


qt.qpa.xcb: QXcbConnection: XCB error: 1 (BadRequest), sequence: 169, resource id: 132, major code: 130 (Unknown), minor code: 47


In [2]:
iline = 0
line = 'Fe_I_6173'
# calib data
config = configobj.ConfigObj('config.ini')
dkdir = config['darks']['directory']
scdir = config['science']['directory']
ffdir = config['flats']['directory']
tgdir = config['targetplate']['directory']
invdir = config['science']['invdir']
#
dqsuffix = '_'
l0dir = config['science'][line+'/l0dir'+dqsuffix]
l0aligndir = config['science'][line+'/l0aligndir'+dqsuffix]
l0corrdir = config['science'][line+'/l0corrdir'+dqsuffix]
#
settings = [f for f in os.listdir(scdir) if 'settings' in f]
settings = scdir + os.sep + settings[0]
settings = configobj.ConfigObj(settings)
#

In [3]:
# Other numbers
iline = get_line_num(settings, line, 0)
linestr = 'Line_' + str(get_line_num(settings, line, iline))
nacc = int(settings[linestr]['Polarimeter\\NAccumulations'])
nwav = int(settings[linestr]['NWavePoints'])
filtstr = settings[linestr]['Filter']
modstr = settings[linestr]['Polarimeter\\Modulation']
nmod = int(settings[filtstr]['Polarimeter\\'+modstr+'\\NModulations'])
nfpc = nmod*nacc*nwav
nfpw = nmod*nacc
ncyc = len(os.listdir(scdir+os.sep+line))//3
wave_range = np.float64(settings[linestr]['WaveScanRange'])
wave_step = wave_range/(np.float64(settings[linestr]['NWavePoints'])-1)
# Time stamps
im0ind, im1ind, im2ind = read_time_stamps_obs(scdir, iline)
im0ind = im0ind.reshape([ncyc,nfpc])
im1ind = im1ind.reshape([ncyc,nfpc])
im2ind = im2ind.reshape([ncyc,nfpc])

#
im0ind = im0ind - im0ind[:,0:1]
im1ind = im1ind - im1ind[:,0:1]
im2ind = im2ind - im2ind[:,0:1]
#
im0ind = im0ind.reshape([ncyc, nwav, nacc, nmod])%nmod
im1ind = im1ind.reshape([ncyc, nwav, nacc, nmod])%nmod
im2ind = im2ind.reshape([ncyc, nwav, nacc, nmod])%nmod

In [ ]:
# dark frames
dk0 = data_cube(dkdir, line, 0, 0)
dk0m = process_dark_cube(dk0.data)

In [ ]:
ff0 = pf.open(config['flats'][line+'/bbi'])
ff0m = ff0[0].data[:,:,np.newaxis] 
ff0m /= ff0m.mean()
ff0.close()

In [ ]:
contrast = []
for i in range(ncyc):
    dc0 = data_cube(scdir, line, 0, i)
    dc0.data = np.float64(dc0.data)
    dc0.data -= dk0m
    dc0.data /= ff0m
    dy, dx = np.gradient(dc0.data, axis=(0,1))
    contr = np.average(np.sqrt(dx**2+dy**2), axis=(0,1))/np.average(dc0.data, axis=(0,1))
    contrast.append(contr)

In [ ]:
i = 121
# bbi
dc0 = data_cube(scdir, line, 0, i)
# pol1
dc1 = data_cube(scdir, line, 1, i)
Y, X, Z = dc1.data.shape
sc1 = dc1.data.reshape([Y,X,nmod,nacc,nwav], order='F')
sc1_add = coadd_del_accumulations (sc1, im1ind[i])
# pol2
dc2 = data_cube(scdir, line, 2, i)
Y, X, Z = dc2.data.shape
sc2 = dc2.data.reshape([Y,X,nmod,nacc,nwav], order='F')
sc2_add = coadd_del_accumulations (sc2, im2ind[i])

In [ ]:
y, x = 1200, 200
fig, ax = plt.subplots(1,2,figsize=(10,5))
ax[0].imshow(sc1_add[:,:,0,0])
ax[1].plot(sc1_add[y,x].T)
fig.tight_layout()

In [ ]:
plt.plot(dc1.data[y,x]-dc1.data[y,x].mean())

In [ ]:
plt.imshow(dc0.data[:,:,10])
plt.tight_layout()
plt.savefig('sc0_bbi.png', dpi=120)

In [ ]:
# Read corrected data
